In [1]:
# !pip install textblob
# !pip install tweepy
# !pip install google.cloud

In [2]:
import tweepy
import re
import os
import pandas as pd
from google.cloud import language
from google.cloud import language_v1
from nltk.tokenize import WordPunctTokenizer
from textblob import TextBlob
import nltk
# nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
# Authentication
apiKey = "mi3IsHw1lzA9KlstGoEi4sQSu"
apiSecret = "0obn2WECJqrZt5eoYJoOY5HbHCQfa5Xe6PFW58EG3AEqF4dxUM"
accessToken = "1441571989357957122-eksbtpbuyFyQiMnIm9t3DOkILvzXLV" 
accessTokenSecret = "aLssavUn0tPln5czeoWaVcf0UYSRYAoenbyBIdf7TPvBE"
auth = tweepy.OAuthHandler(apiKey, apiSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [4]:
# Instantiates a client
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/path/to/file.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/zjw64/Desktop/ec601/project2/service-account-file.json"
client = language_v1.LanguageServiceClient()

In [14]:
#search tweets
keyword = input("Please enter keyword to search: ")
numOfTweet = int(input ("Please enter the number of tweets to analyze: "))

tweets = tweepy.Cursor(api.search_tweets, q=keyword).items(numOfTweet)
tweet_list = []
for tweet in tweets:
    tweet_list.append(tweet.text)

tweet_list = pd.DataFrame(tweet_list)
print("search number: ",len(tweet_list))

In [15]:
# delete duplicates
tweet_list.drop_duplicates(inplace = True)
print("total number: ",len(tweet_list))

#Extracting text values
tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]
print(tw_list.head(10))

In [16]:
#Cleaning Text (RT, Punctuation etc)

#Creating new dataframe and new features
tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]

#Removing RT, Punctuation etc
remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
tw_list["text"] = tw_list.text.map(remove_rt).map(rt)
tw_list["text"] = tw_list.text.str.lower()
print(tw_list.head(10))

In [17]:
#Twitter sentiment analyze
tw_twi = tw_list.copy(deep=True)
tw_twi[['polarity', 'subjectivity']] = tw_twi['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tw_twi['text'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
        tw_twi.loc[index, 'sentiment'] = "negative"
    elif pos > neg:
        tw_twi.loc[index, 'sentiment'] = "positive"
    else:
        tw_twi.loc[index, 'sentiment'] = "neutral"
    tw_twi.loc[index, 'neg'] = neg
    tw_twi.loc[index, 'neu'] = neu
    tw_twi.loc[index, 'pos'] = pos
    tw_twi.loc[index, 'compound'] = comp

print(tw_twi.head(10))

#Creating new data frames for all sentiments (positive, negative and neutral)
tw_twi_negative = tw_twi[tw_twi["sentiment"]=="negative"]
tw_twi_positive = tw_twi[tw_twi["sentiment"]=="positive"]
tw_twi_neutral = tw_twi[tw_twi["sentiment"]=="neutral"]

In [18]:
# Google NLP API sentiment analyze
tw_goo = tw_list.copy(deep=True)
for index, tweet in tw_goo['text'].iteritems():
    document = language_v1.Document(content=tweet.encode('utf-8'), type_=language_v1.Document.Type.PLAIN_TEXT)
    sentiment = client.analyze_sentiment(request={'document': document}).document_sentiment
    tw_goo.loc[index, 'score'] = sentiment.score

    if sentiment.score >= -1.0 and sentiment.score < -0.25:
        tw_goo.loc[index, 'sentiment'] = "negative"
    elif sentiment.score >= -0.25 and sentiment.score <= 0.25:
        tw_goo.loc[index, 'sentiment'] = "neutral"
    else:
        tw_goo.loc[index, 'sentiment'] = "positive"

print(tw_goo.head(10))


#Creating new data frames for all sentiments (positive, negative and neutral)

tw_goo_negative = tw_goo[tw_goo["sentiment"]=="negative"]
tw_goo_positive = tw_goo[tw_goo["sentiment"]=="positive"]
tw_goo_neutral = tw_goo[tw_goo["sentiment"]=="neutral"]

In [13]:
#Function for count_values_in single columns

def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

#Count_values for sentiment
print("Twitter API sentiment analyze: ")
print(count_values_in_column(tw_twi,"sentiment"))
print("Google NLP sentiment analyze: ")
print(count_values_in_column(tw_goo,"sentiment"))

Twitter API sentiment analyze: 
          Total  Percentage
neutral     273       65.16
positive     95       22.67
negative     51       12.17
Google NLP sentiment analyze: 
          Total  Percentage
NaN         363       86.63
neutral      32        7.64
positive     15        3.58
negative      9        2.15
